<a href="https://colab.research.google.com/github/NataHsH/LLM_tiny-shakespeare-transformer/blob/main/LanguageModel_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install required libraries
# ==========================
!pip install transformers datasets wandb accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [31]:
!pip install huggingface_hub


In [4]:
# Login to Hugging Face and Weights & Biases
# ==========================
from huggingface_hub import notebook_login
import wandb

# Login to Hugging Face
notebook_login()

# Login to Weights & Biases
wandb.login()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


True

In [3]:
# Import necessary libraries
# ==========================
import torch
from torch import nn
from torch.utils.data import DataLoader
from transformers import PreTrainedTokenizerFast
from datasets import load_dataset
import wandb
from huggingface_hub import HfApi, create_repo, upload_folder


In [6]:
# Define hyperparameters
# ==========================
batch_size = 64
block_size = 128
embedding_dim = 256
num_heads = 4
num_layers = 4
dropout = 0.1
learning_rate = 3e-4
num_epochs = 3

model_name = "tiny_shakespeare_transformer"
repository_id = "NataliaH/tiny_shakespeare_transformer"

In [7]:
# Load dataset
# ==========================
dataset = load_dataset("tiny_shakespeare", split="train")
text = dataset['text']

README.md:   0%|          | 0.00/6.10k [00:00<?, ?B/s]

tiny_shakespeare.py:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

The repository for tiny_shakespeare contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/tiny_shakespeare.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/1 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1 [00:00<?, ? examples/s]

In [8]:
# Tokenizer
# ==========================
# Create and train a simple character-level tokenizer
tokenizer = PreTrainedTokenizerFast.from_pretrained("gpt2")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Encode the dataset
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=block_size)
input_ids = inputs["input_ids"]


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [109]:
# Prepare the dataset for training
# ==========================
class ShakespeareDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, block_size):
        self.input_ids = input_ids
        self.block_size = block_size

    def __len__(self):
        return self.input_ids.size(1) // self.block_size

    def __getitem__(self, idx):
        start_idx = idx * self.block_size
        end_idx = start_idx + self.block_size
        x = self.input_ids[:, start_idx:end_idx]
        y = self.input_ids[:, start_idx+1:end_idx+1]
        return x.squeeze(0), y.squeeze(0)

dataset = ShakespeareDataset(input_ids, block_size)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [110]:
# Prepare the dataset
def group_texts(tokenized_text, block_size):
    total_length = len(tokenized_text)
    total_length = (total_length // block_size) * block_size  # truncate to a multiple of block_size
    input_ids = tokenized_text[:total_length]

    input_ids = input_ids.view(-1, block_size)
    target_ids = input_ids.clone()
    return input_ids, target_ids

# Tokenize the entire dataset
tokenized_text = tokenizer(text, return_tensors='pt', add_special_tokens=False).input_ids.squeeze(0)

# Group into blocks
block_size = 128
x, y = group_texts(tokenized_text, block_size)


In [125]:
from transformers import AutoTokenizer
import torch
import json

# Load the model state dict
model_save_path = "./tiny_shakespeare_transformer"
tokenizer_save_path = model_save_path

# Define the model architecture
class TransformerModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_heads, num_layers, block_size, dropout):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, embedding_dim)
        decoder_layer = nn.TransformerDecoderLayer(
            d_model=embedding_dim,
            nhead=num_heads,
            dim_feedforward=embedding_dim * 4,
            dropout=dropout,
            activation='gelu'
        )
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_layers)
        self.fc_out = nn.Linear(embedding_dim, vocab_size)

    def forward(self, x):
        embedded = self.token_embedding(x)
        memory = torch.zeros_like(embedded)
        out = self.transformer_decoder(embedded, memory)
        logits = self.fc_out(out)
        return logits

# Load the model state dict
model = TransformerModel(
    vocab_size=5000,  # Example vocab size, adjust based on your model
    embedding_dim=256,  # Example, adjust as needed
    num_heads=8,  # Example
    num_layers=6,  # Example
    block_size=128,  # Example
    dropout=0.1  # Example
)

# Load model weights and move to the correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load(f"{model_save_path}/model.pth"))
model.to(device)
model.eval()

# Load tokenizer
with open(f"{tokenizer_save_path}/tokenizer.json", "r") as f:
    vocab = json.load(f)

tokenizer = AutoTokenizer.from_pretrained("gpt2")  # Or adjust to your model type
tokenizer.add_tokens(list(vocab.keys()))  # Adding custom vocab

# Ensure tokenizer vocab size is not larger than model vocab size
tokenizer_vocab_size = len(tokenizer)
model_vocab_size = model.token_embedding.num_embeddings
if tokenizer_vocab_size > model_vocab_size:
    tokenizer = AutoTokenizer.from_pretrained("gpt2", use_fast=False)
    tokenizer.add_tokens(list(vocab.keys())[:model_vocab_size])

# Add pad token if necessary
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


In [126]:
# Define optimizer and loss function
# ==========================
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# Initialize wandb
wandb.init(project="LanguageModel_Project", name="tiny_shakespeare_transformer")


In [13]:
import time
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

# Move the data to the same device as the model
x_train = x_train.to(device)
y_train = y_train.to(device)
x_val = x_val.to(device)
y_val = y_val.to(device)

# Training loop
# ==========================
num_epochs = 5
model.train()

for epoch in range(num_epochs):
    start_time = time.time()  # Track the time taken for the epoch
    total_train_loss = 0

    # Loop through the dataset manually in batches
    for i in range(0, x_train.size(0), batch_size):
        xb = x_train[i:i+batch_size].to(device)
        yb = y_train[i:i+batch_size].to(device)

        # Skip the last batch if it's smaller than batch_size
        if xb.size(0) != batch_size or yb.size(0) != batch_size:
            continue

        optimizer.zero_grad()               # Reset gradients
        logits = model(xb)                   # Forward pass
        loss = criterion(logits.view(-1, logits.size(-1)), yb.view(-1))  # Compute loss
        loss.backward()                      # Backward pass
        optimizer.step()                     # Update weights

        total_train_loss += loss.item()

    # Validation step
    model.eval()  # Switch the model to evaluation mode
    total_val_loss = 0
    with torch.no_grad():  # No need to compute gradients during validation
        for i in range(0, x_val.size(0), batch_size):
            xb_val = x_val[i:i+batch_size].to(device)
            yb_val = y_val[i:i+batch_size].to(device)

            # Skip the last batch if it's smaller than batch_size
            if xb_val.size(0) != batch_size or yb_val.size(0) != batch_size:
                continue

            logits_val = model(xb_val)  # Forward pass
            val_loss = criterion(logits_val.view(-1, logits_val.size(-1)), yb_val.view(-1))  # Compute loss
            total_val_loss += val_loss.item()

    # Compute average losses for the epoch
    num_batches_train = x_train.size(0) // batch_size
    avg_train_loss = total_train_loss / num_batches_train if num_batches_train > 0 else 0

    num_batches_val = x_val.size(0) // batch_size
    avg_val_loss = total_val_loss / num_batches_val if num_batches_val > 0 else 0

    # Get the current learning rate
    current_lr = optimizer.param_groups[0]['lr']

    # Calculate the time taken for the epoch
    epoch_time = time.time() - start_time

    # Log all metrics
    wandb.log({
        "train_loss": avg_train_loss,
        "val_loss": avg_val_loss,
        "learning_rate": current_lr,
        "epoch_time": epoch_time
    }, step=epoch)

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, LR: {current_lr:.6f}, Time: {epoch_time:.2f}s")


Epoch 1/5, Train Loss: 7.3459, Val Loss: 5.1880, LR: 0.000300, Time: 787.37s
Epoch 2/5, Train Loss: 4.0344, Val Loss: 3.1616, LR: 0.000300, Time: 761.04s
Epoch 3/5, Train Loss: 2.5472, Val Loss: 2.1426, LR: 0.000300, Time: 746.07s
Epoch 4/5, Train Loss: 1.7511, Val Loss: 1.5593, LR: 0.000300, Time: 746.92s
Epoch 5/5, Train Loss: 1.2723, Val Loss: 1.1928, LR: 0.000300, Time: 746.79s


In [127]:
import os
import torch
import json
from transformers import AutoTokenizer

# Define the save path
model_save_path = "./tiny_shakespeare_transformer"
tokenizer_save_path = model_save_path

# Create directories if they don't exist
os.makedirs(model_save_path, exist_ok=True)

# Save model state dict
torch.save(model.state_dict(), f"{model_save_path}/model.pth")

# Save tokenizer (updated to handle custom tokenizer correctly)
# Assuming you already added custom tokens and made necessary adjustments to the tokenizer
tokenizer.save_pretrained(tokenizer_save_path)

# If you want to save the vocabulary separately, you can also do it
with open(f"{tokenizer_save_path}/tokenizer.json", "w") as f:
    json.dump(tokenizer.get_vocab(), f)


In [128]:
# Push model to Hugging Face Hub
# ==========================
api = HfApi()
create_repo(repo_id=repository_id, exist_ok=True)
upload_folder(folder_path=model_save_path, repo_id=repository_id)

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/NataliaH/tiny_shakespeare_transformer/commit/0957da3a15ca020a8e46d6404e6615fbb0ae9e7c', commit_message='Upload folder using huggingface_hub', commit_description='', oid='0957da3a15ca020a8e46d6404e6615fbb0ae9e7c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/NataliaH/tiny_shakespeare_transformer', endpoint='https://huggingface.co', repo_type='model', repo_id='NataliaH/tiny_shakespeare_transformer'), pr_revision=None, pr_num=None)

In [130]:
import os
import textwrap

# Define the path to save model and README
model_save_path = "./tiny_shakespeare_transformer"

# Create Model Card
# ==========================
model_card = textwrap.dedent(f"""\
    ---
    license: mit
    tags:
    - text-generation
    - transformer
    - tiny-shakespeare
    - decoder-only
    model-index:
    - name: tiny_shakespeare_transformer
      results: []
    ---

    # tiny_shakespeare_transformer

    A small Transformer Decoder model trained from scratch on the Tiny Shakespeare dataset.

    ## Training details
    - Dataset: Tiny Shakespeare
    - Epochs: {num_epochs}
    - Learning Rate: {learning_rate}
    - Batch Size: {batch_size}
    - Block Size: {block_size}
    - Optimizer: AdamW
    - Loss Function: CrossEntropyLoss
    - Dropout Rate: {dropout}
    - Embedding Dimension: {embedding_dim}
    - Number of Layers: {num_layers}
    - Number of Attention Heads: {num_heads}

    ## Usage
    To use this model, simply load it using the following code:

    ```python
    from transformers import AutoModelForCausalLM, AutoTokenizer

    # Load the model and tokenizer
    model = AutoModelForCausalLM.from_pretrained("{repository_id}")
    tokenizer = AutoTokenizer.from_pretrained("{repository_id}")

    # Encode input text
    inputs = tokenizer("Once upon a time", return_tensors="pt")
    outputs = model.generate(**inputs)
    print(tokenizer.decode(outputs[0]))
    ```

    ## Model Architecture
    This model is a Transformer Decoder-based architecture, optimized for text generation.
    It was trained on the Tiny Shakespeare dataset to generate Shakespeare-like text.

    ## Training Process
    - Training was performed for {num_epochs} epochs.
    - The model uses AdamW optimizer with a learning rate of {learning_rate}.
    - Dropout rate during training was set to {dropout} to reduce overfitting.

    ## License
    This model is released under the MIT License.
""")

# Save the model card to README.md
os.makedirs(model_save_path, exist_ok=True)

with open(f"{model_save_path}/README.md", "w") as f:
    f.write(model_card)

# Print the contents of the README to verify
!cat {model_save_path}/README.md


---
license: mit
tags:
- text-generation
- transformer
- tiny-shakespeare
- decoder-only
model-index:
- name: tiny_shakespeare_transformer
  results: []
---

# tiny_shakespeare_transformer

A small Transformer Decoder model trained from scratch on the Tiny Shakespeare dataset.

## Training details
- Dataset: Tiny Shakespeare
- Epochs: 5
- Learning Rate: 0.0003
- Batch Size: 32
- Block Size: 128
- Optimizer: AdamW
- Loss Function: CrossEntropyLoss
- Dropout Rate: 0.1
- Embedding Dimension: 256
- Number of Layers: 6
- Number of Attention Heads: 8

## Usage
To use this model, simply load it using the following code:

```python
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained("NataliaH/tiny_shakespeare_transformer")
tokenizer = AutoTokenizer.from_pretrained("NataliaH/tiny_shakespeare_transformer")

# Encode input text
inputs = tokenizer("Once upon a time", return_tensors="pt")
outputs = model.generate(

In [131]:
from huggingface_hub import upload_folder

# Параметры
repo_id = "NataliaH/tiny_shakespeare_transformer"  # Название репозитория, который уже существует
model_save_path = "./tiny_shakespeare_transformer"

# Загрузить модель в репозиторий
upload_folder(
    folder_path=model_save_path,
    repo_id=repo_id,
    commit_message="Initial commit with model and tokenizer",
)


No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/NataliaH/tiny_shakespeare_transformer/commit/0957da3a15ca020a8e46d6404e6615fbb0ae9e7c', commit_message='Initial commit with model and tokenizer', commit_description='', oid='0957da3a15ca020a8e46d6404e6615fbb0ae9e7c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/NataliaH/tiny_shakespeare_transformer', endpoint='https://huggingface.co', repo_type='model', repo_id='NataliaH/tiny_shakespeare_transformer'), pr_revision=None, pr_num=None)

In [132]:
import torch

# Function for text generation manually
def generate_text_manually(model, tokenizer, prompt, max_length=300):
    # Tokenize the input prompt
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    generated_ids = input_ids

    # Text generation loop
    for _ in range(max_length):
        # Pass the generated ids through the model
        output = model(generated_ids)  # Model output is a tensor of logits
        logits = output[:, -1, :]  # Only take the last token's logits (for the last token in the sequence)

        # Get the most probable next token
        next_token_id = torch.argmax(logits, dim=-1).unsqueeze(-1)  # Get the token with the highest probability

        # Append the new token to the generated tokens
        generated_ids = torch.cat((generated_ids, next_token_id), dim=-1)

        # Stop if the end of sequence token is generated
        if next_token_id.item() == tokenizer.eos_token_id:
            break

    # Decode the generated tokens back to text, skip special tokens and clean the output
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    generated_text = generated_text.replace('Ġ', ' ')  # Replace Ġ with spaces
    return generated_text

# Example usage
prompt = "Once upon a time"
generated_text = generate_text_manually(model, tokenizer, prompt, max_length=300)


print(generated_text)


On ce   up on   a   time ended  ste 47 03 l  blood Qu  Blue ational  Hill edom der  sexual  steps  note  wa ten ny  kept dom ï¿½ ic Ī pped aving  24  reach ____ © § mer  screen  fem  taking  football acebook orge  fit  Mon ur  hundred  en stem  function  allowed tern  lim iam red  hum  International  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  file  show  fi

In [152]:

import json

# Путь к твоему ноутбуку (замени на путь к твоему файлу)
notebook_file = '/content/drive/MyDrive/Colab Notebooks/LanguageModel_Project.ipynb'  # Убедись, что путь правильный

# Открываем файл и загружаем содержимое
with open(notebook_file, 'r') as f:
    notebook = json.load(f)

# Удаляем раздел 'widgets' из 'metadata'
if 'metadata' in notebook:
    if 'widgets' in notebook['metadata']:
        del notebook['metadata']['widgets']

# Сохраняем изменения в файл
with open(notebook_file, 'w') as f:
    json.dump(notebook, f, indent=4)

print(f"Widgets successfully removed from {notebook_file}")


Widgets successfully removed from /content/drive/MyDrive/Colab Notebooks/LanguageModel_Project.ipynb
